In [1]:
getwd()

[1] "C:/Users/rpgen/Documents/CS424/project3/bigYellowTaxi"

In [ ]:
#READ CSV FILE AND CONVERT TO ONE DATA FRAME
col_name <- c("","Trip.Start.Timestamp","Trip.Seconds","Trip.Miles","Pickup.Community.Area","Dropoff.Community.Area","Company")
setwd("/Users/rpgen/Documents/CS424/project3/bigYellowTaxi/Data")
myfiles <- list.files(pattern="*.csv", full.names=TRUE)
myfiles
data <- do.call(rbind, lapply(myfiles, read.csv, header = FALSE))
colnames(data) <- col_name



[1] "./by_hour.csv"              "./modified_taxi_partaa.csv"
 [3] "./modified_taxi_partab.csv" "./modified_taxi_partac.csv"
 [5] "./modified_taxi_partad.csv" "./modified_taxi_partae.csv"
 [7] "./modified_taxi_partaf.csv" "./modified_taxi_partag.csv"
 [9] "./modified_taxi_partah.csv" "./modified_taxi_partai.csv"
[11] "./modified_taxi_partaj.csv" "./modified_taxi_partak.csv"
[13] "./modified_taxi_partal.csv" "./modified_taxi_partam.csv"
[15] "./modified_taxi_partan.csv" "./modified_taxi_partao.csv"
[17] "./modified_taxi_partap.csv" "./modified_taxi_partaq.csv"

In [ ]:
data

In [ ]:
library(lubridate)

n_date <- parse_date_time(data$Trip.Start.Timestamp,
                orders = 'mdY IMS %p', truncated = 3) #PARSE DATE FROM TIME STAMP
data$new_date <- n_date
data$Hour <- hour(data$new_date)
data

In [ ]:
#Aggregate by day
noFilter_byDay <- aggregate(data[,1], by=list(date(data$new_date)), FUN=length)
colnames(noFilter_byDay) <- c("Date", "Rides")
noFilter_byDay

In [ ]:
#Aggregate by hour
noFilter_byHour <- aggregate(data[,1], by=list(data$Hour), FUN=length)
colnames(noFilter_byHour) <- c("Hour", "Rides")
noFilter_byHour

In [ ]:
#Aggregate by weekday
noFilter_byDayOfWeek <- aggregate(data[,1], by=list(wday(data$new_date, week_start=1)), FUN=length)
colnames(noFilter_byDayOfWeek) <- c("Day Of Week", "Rides")
#** STARTS MONDAY TO SUNDAY WITH 1 BEING MONDAY **
noFilter_byDayOfWeek

In [ ]:
#Aggregate by binned mileage
print(min(data$Trip.Miles))
max(data$Trip.Miles)


In [ ]:
sort(unique(data$Pickup.Community.Area))

In [ ]:
community_list <- c("Rogers Park", "West Ridge", "Uptown", 
          "Lincoln Square", "North Center", "Lake View",
         "Lincoln Park", "Near North Side", "Edison Park",
         "Norwood Park", "Jefferson Park", "Forest Glen",
         "North Park", "Albany Park", "Portage Park",
         "Irving Park", "Dunning", "Montclare", "Belmont Cragin",
         "Hermosa", "Avondale", "Logan Square", "Humboldt Park",
         "West Town", "Austin", "West Garfield Park",
         "East Garfield Park", "Near West Side", "North Lawndale",
         "South Lawndale", "Lower West Side", "Loop", "Near South Side",
         "Armour Square", "Douglas", "Oakland", "Fuller Park", 
         "Grand Boulevard", "Kenwood", "Washington Park", "Hyde Park",
         "Woodlawn", "South Shore", "Chatham", "Avalon Park", "South Chicago",
         "Burnside", "Calumet Heights", "Roseland", "Pullman", "South Deering",
         "East Side", "West Pullman", "Riverdale", "Hegewisch", "Garfield Ridge",
         "Archer Heights", "Brighton Park", "Mckinley Park", "Bridgeport",
         "New City", "West Elsdon", "Gage Park", "Clearing", "West Lawn",
         "Chicago Lawn", "West Englewood", "Englewood", "Greater Grand Crossing",
         "Ashburn", "Auburn Gresham", "Beverly", "Washington Heights",
         "Mount Greenwood", "Morgan Park", "Ohare", "Edgewater") # R IS INDEX STARTING 1 NOT 0

sorted_community_list <- sort(community_list)
new_sorted_community_list <- append(sorted_community_list, "All (City of Chicago)", 0)
new_sorted_community_list

In [ ]:
library(rlang)
library(ggplot2)

In [ ]:
noFilter_byDay <- aggregate(data[,1], by=list(date(data$new_date)), FUN=length)
colnames(noFilter_byDay) <- c("Date", "Rides")
noFilter_byDay

In [ ]:
ggplot(noFilter_byDay, aes(x=`Date`, y=`Rides`)) +
  geom_bar(stat="identity")

In [ ]:
# Dataframe for the percentage of rides going to/from each community
# NEEDS TO ACCOUNT THE TO/FROM FILTER
# INTEGRATE WITH TAXI
# KEEP IN MIND TO KEEP THE ORDER OF COMMUNITY ARRAY
eachCommunity <- data.frame(data)
comm_range <- 1:77
company <- "All Taxis"
trip <- "To"

if(company != "All Taxis"){
    eachCommunity <- subset(eachCommunity, eachCommunity$Company == company)
}

if(trip == "To"){
    eachCommunity <- aggregate(eachCommunity[,1], by=list(eachCommunity$Dropoff.Community.Area), FUN=length)
}else{
    eachCommunity <- aggregate(eachCommunity[,1], by=list(eachCommunity$Pickup.Community.Area), FUN=length)
}

colnames(eachCommunity) <- c("Community", "Percentage")
#Fill in missing communities with 0's
for(i in 1:77){
    hasCommunity <- FALSE
    for(j in 1:length(eachCommunity[,1])){
        if(eachCommunity[j,1] == i){
            hasCommunity <- TRUE
            break
        }
    }
        
    if(!hasCommunity){
        eachCommunity[nrow(eachCommunity) + 1,] = c(i, 0)
    }
}

eachCommunity <- eachCommunity[order(eachCommunity$Community),]
rownames(eachCommunity) <- 1:nrow(eachCommunity)

#CONVERT COMMUNITY ID TO NAMES
for(i in 1:77){
    eachCommunity[i,1] <- community_list[i]
}

#Table them as percentages
count_sum <- sum(eachCommunity$Percentage)
eachCommunity$Percentage <- eachCommunity$Percentage/count_sum
eachCommunity <- eachCommunity[order(eachCommunity$Community),, drop=FALSE]
eachCommunity
#NOTE BAR CHART NEEDS TO SHOW COMMUNITIES ALPHABETICALLY

In [ ]:
# sort_comm <- sort(eachCommunity$Community)
# sort_comm

sorting_com <- eachCommunity[order(eachCommunity$Community),, drop=FALSE]
sorting_com

In [ ]:
ggplot(sorting_com, aes(x=`Community`, y=`Percentage`)) +
  geom_bar(stat="identity") + theme(axis.text.x = element_text(angle = 90))

In [ ]:
noFilter_byHour <- aggregate(data[,1], by=list(data$Hour), FUN=length)
colnames(noFilter_byHour) <- c("Hour", "Rides")
write.csv(noFilter_byHour, "by_hour.csv")

In [ ]:
noFilter_byMonth <- aggregate(data[,1], by=list(month(data$new_date)), FUN=length)
colnames(noFilter_byMonth) <- c("Month", "Rides")
write.csv(noFilter_byMonth, "/Users/rpgen/Documents/CS424/project3/bigYellowTaxi/by_month.csv")